# Structured Streaming

In [2]:
# Load the data
staticDataFrame = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/databricks-datasets/definitive-guide/data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

In [3]:
# Create the daily spends column
from pyspark.sql.functions import window, column, desc, col

staticDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")\
  .sort(desc("sum(total_cost)"))\
  .show(5)

+----------+--------------------+------------------+
CustomerId| window| sum(total_cost)|
+----------+--------------------+------------------+
 17450.0|[2011-09-20 00:00...| 71601.44|
 null|[2011-11-14 00:00...| 55316.08|
 null|[2011-11-07 00:00...| 42939.17|
 null|[2011-03-29 00:00...| 33521.39999999998|
 null|[2011-12-08 00:00...|31975.590000000007|
+----------+--------------------+------------------+
only showing top 5 rows

In [4]:
# Reduce the number of partitions since there are not too many workers
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Read the data in Stream
streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1)\
    .format("csv")\
    .option("header", "true")\
    .load("/data/retail-data/by-day/*.csv")

# Check the streaming
streamingDataFrame.isStreaming

Out[ 16 ]: True

In [5]:
# Create again the daily spends column
purchaseByCustomerPerHour = streamingDataFrame\
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")\
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
  .sum("total_cost")

In [6]:
# Perform and action
purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()

Out[ 18 ]: <pyspark.sql.streaming.StreamingQuery at 0x7f88feb0a470>

In [7]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)\
  .show(5)

+----------+------+---------------+
CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+

# Machine Learning and Advanced Analytics

In [9]:
# Work out the missing values
from pyspark.sql.functions import date_format, col

preppedDataFrame = staticDataFrame\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
  .coalesce(5)

In [10]:
# Create a train and a test set
trainDataFrame = preppedDataFrame\
  .where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
  .where("InvoiceDate >= '2011-07-01'")

In [11]:
# Give days numerical values
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer()\
  .setInputCol("day_of_week")\
  .setOutputCol("day_of_week_index")

In [12]:
# Break the order by setting a categorical variable
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder()\
  .setInputCol("day_of_week_index")\
  .setOutputCol("day_of_week_encoded")

In [13]:
# Assemble all features into a vector
from pyspark.ml.feature import VectorAssembler

vectorAssembler = VectorAssembler()\
  .setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
  .setOutputCol("features")

In [14]:
# Set up a pipeline
from pyspark.ml import Pipeline

transformationPipeline = Pipeline()\
  .setStages([indexer, encoder, vectorAssembler])

In [15]:
# Fit and transform the data through the pipeline
fittedPipeline = transformationPipeline.fit(trainDataFrame)
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [16]:
# Cache the DataFrame to improve times
transformedTraining.cache()

Out[ 27 ]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: timestamp, UnitPrice: double, CustomerID: double, Country: string, day_of_week: string, day_of_week_index: double, day_of_week_encoded: vector, features: vector]

In [17]:
# Instantiate the model and make it work
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
  .setK(20)\
  .setSeed(1)

kmModel = kmeans.fit(transformedTraining)

In [18]:
# Use the model to make predictions

transformedTest = fittedPipeline.transform(testDataFrame)
kmModel.computeCost(transformedTest)

Out[ 38 ]: 517507094.72221166

# Lower-Level APIs

In [20]:
# Transform a RDD into a DataFrame
from pyspark.sql import Row

spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()

Out[ 39 ]: DataFrame[_1: bigint]